- 단계구분도: 경기도내 인구소멸비율

In [1]:
import json,folium
import numpy as np
import pandas as pd

- 경기도 시군구별 인구소멸비율 구하기

In [2]:
pop = pd.read_csv('../02.Pandas/data/주민등록인구집계현황.csv', encoding='euc-kr')
pop = pop[['행정구역구분명','행정구역명','총 인구수','60~69세','70~79세','80~89세','90~99세','100세 이상','20~29세 (여)','30~39세 (여)']]
pop['인구소멸비율'] = (pop.iloc[:, -2:].sum(axis=1) / pop.iloc[:, 2:7].sum(axis=1)).round(3)
pop = pop[['행정구역구분명','행정구역명','총 인구수','인구소멸비율']]
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
0,도,경기도,13589432,0.104
1,시군,경기도 가평군,62197,0.057
2,읍면동,경기도 가평군 가평읍,19555,0.067
3,읍면동,경기도 가평군 북면,3815,0.032
4,읍면동,경기도 가평군 상면,5669,0.037


In [3]:
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
1,시군,경기도 가평군,62197,0.057
8,시군,경기도 고양시,1075202,0.106
9,구,경기도 고양시 덕양구,487874,0.107
31,구,경기도 고양시 일산동구,296590,0.110
44,구,경기도 고양시 일산서구,290738,0.102


In [4]:
pop

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
1,시군,경기도 가평군,62197,0.057
8,시군,경기도 고양시,1075202,0.106
9,구,경기도 고양시 덕양구,487874,0.107
31,구,경기도 고양시 일산동구,296590,0.110
44,구,경기도 고양시 일산서구,290738,0.102
56,시군,경기도 과천시,77775,0.112
63,시군,경기도 광명시,288182,0.100
83,시군,경기도 광주시,391704,0.097
96,시군,경기도 광주시 오포2동,34293,0.100
100,시군,경기도 구리시,188876,0.102


In [5]:
gu_city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pop.행정구역명 = pop.행정구역명.str.strip()
drop_index = []
for index in pop[pop.행정구역구분명 == '시군'].index:
    if pop.행정구역명[index].split()[-1] in gu_city:
        drop_index.append(index)
pop.drop(drop_index, inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
1,시군,경기도 가평군,62197,0.057
9,구,경기도 고양시 덕양구,487874,0.107
31,구,경기도 고양시 일산동구,296590,0.110
44,구,경기도 고양시 일산서구,290738,0.102
56,시군,경기도 과천시,77775,0.112


- geo data 읽기

In [6]:
filename = 'data/seoul_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

- 인구 데이터와 geo 데이터간 인덱스 일치 시키기

In [7]:
pop['시군구'] = pop.행정구역명.apply(lambda x: ''.join(x.split()[1:]))
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율,시군구
1,시군,경기도 가평군,62197,0.057,가평군
9,구,경기도 고양시 덕양구,487874,0.107,고양시덕양구
31,구,경기도 고양시 일산동구,296590,0.110,고양시일산동구
44,구,경기도 고양시 일산서구,290738,0.102,고양시일산서구
56,시군,경기도 과천시,77775,0.112,과천시


In [8]:
pop.set_index('시군구', inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
시군구,,,,
가평군,시군,경기도 가평군,62197,0.057
고양시덕양구,구,경기도 고양시 덕양구,487874,0.107
고양시일산동구,구,경기도 고양시 일산동구,296590,0.110
고양시일산서구,구,경기도 고양시 일산서구,290738,0.102
과천시,시군,경기도 과천시,77775,0.112


In [10]:
pop

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
시군구,,,,
가평군,시군,경기도 가평군,62197,0.057
고양시덕양구,구,경기도 고양시 덕양구,487874,0.107
고양시일산동구,구,경기도 고양시 일산동구,296590,0.110
고양시일산서구,구,경기도 고양시 일산서구,290738,0.102
과천시,시군,경기도 과천시,77775,0.112
광명시,시군,경기도 광명시,288182,0.100
광주시,시군,경기도 광주시,391704,0.097
광주시오포2동,시군,경기도 광주시 오포2동,34293,0.100
구리시,시군,경기도 구리시,188876,0.102


In [9]:
map = folium.Map([37.4, 127.1], zoom_start=9, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=pop.인구소멸비율,      # 단계구분도로 보여줄 데이터
    columns=[pop.index, pop.인구소멸비율],    # 데이터프레임에서 추출할 항목
    fill_color='YlOrRd',        # Colormap
    key_on='properties.name'    # 지도에서 조인할 항목
).add_to(map)
map